In [ ]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

## 1. Data Preprocessing-stock price&movement

In [ ]:
stock = pd.read_csv('2603_stockprice.csv')

choosing "EVERGREEN" (2603) as the targeted company

In [ ]:
ase = stock[stock['證券代碼'].str.contains('2603')]
ase['年月日'] = pd.to_datetime(ase['年月日']).dt.date
ase = ase.sort_values(by = '年月日').reset_index(drop = True)

In [ ]:
ase = ase[['證券代碼', '年月日', '收盤價(元)']]
ase['單日漲跌幅(%)'] = ase['收盤價(元)'].rolling(window=2).apply(lambda x: (x.iloc[1]-x.iloc[0])/x.iloc[0])
ase['label'] = ase['單日漲跌幅(%)'].apply(lambda x: (x>0 and '漲') or '跌')

In [ ]:
ase

,證券代碼,年月日,收盤價(元),單日漲跌幅(%),label
0,2603 長榮,2022-01-03,269.2919,NaN,跌
1,2603 長榮,2022-01-04,271.2223,0.007168,漲
2,2603 長榮,2022-01-05,272.1875,0.003559,漲
3,2603 長榮,2022-01-06,276.0483,0.014184,漲
4,2603 長榮,2022-01-07,271.2223,-0.017482,跌
...,...,...,...,...,...
290,2603 長榮,2023-03-20,162.0000,0.000000,跌
291,2603 長榮,2023-03-21,160.5000,-0.009259,跌
292,2603 長榮,2023-03-22,161.5000,0.006231,漲
293,2603 長榮,2023-03-23,159.5000,-0.012384,跌


In [ ]:
ase['label'].value_counts()

跌    155
漲    140
Name: label, dtype: int64

## 2. Data Preprocessing - text data

In [ ]:
# 讀取三份CSV檔案
import pandas as pd
usecols=['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title','author','content', 'page_url']

df1 = pd.read_csv('/Users/dora/Desktop/大數據與商業分析/Data/bda2023_mid_bbs_2022-2023.csv',usecols=usecols)
df2 = pd.read_csv('/Users/dora/Desktop/大數據與商業分析/Data/bda2023_mid_forum_2022-2023.csv', usecols=usecols)
df3 = pd.read_csv('/Users/dora/Desktop/大數據與商業分析/Data/bda2023_mid_news_2022-2023.csv', usecols=usecols)

In [ ]:
# 使用concat函數合併三個DataFrame
news23 = pd.concat([df1, df2, df3], ignore_index=True)

# 將日期欄位轉成 datetime
news23['post_time'] = pd.to_datetime(news23['post_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# 顯示合併後的DataFrame
news23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447542 entries, 0 to 447541
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           447542 non-null  object        
 1   p_type       447542 non-null  object        
 2   s_name       447542 non-null  object        
 3   s_area_name  447542 non-null  object        
 4   post_time    447540 non-null  datetime64[ns]
 5   title        447542 non-null  object        
 6   author       420716 non-null  object        
 7   content      445823 non-null  object        
 8   page_url     447539 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 30.7+ MB


In [ ]:
news23_ase = news23[news23['title'].str.contains('長榮') | news23['content'].str.contains('長榮')].reset_index(drop = True)
news23_ase

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1641004905375_PTT02R,bbs,Ptt,Stock,2022-01-01 10:38:00,[新聞] 長榮超狂年終有所本！SCFI封關飆破5000點,brella,新聞標題：長榮超狂年終有所本！SCFI封關飆破 5000點 歐美線續居天...,http://www.ptt.cc/bbs/Stock/M.1641004690.A.3CE...
1,1641010550891_PTT02R,bbs,Ptt,Stock,2022-01-01 12:08:00,[新聞] 夢幻長榮 內行爆2024運能暴增恐是拐點,JessVeron,原文標題：夢幻長榮 內行爆2024運能暴增恐是拐點 ...,http://www.ptt.cc/bbs/Stock/M.1641010114.A.E3B...
2,1641011258259_PTT02R,bbs,Ptt,Stock,2022-01-01 12:22:00,[心得] 2021總結,jaslyn,2021結束了，大部分的板友都有不錯的成績吧? 以下是小的今年的已實現損益： https:/...,http://www.ptt.cc/bbs/Stock/M.1641010927.A.FFF...
3,1641018691102_PTT02R,bbs,Ptt,Stock,2022-01-01 14:28:00,[新聞] 全球塞船運費飆漲 估恐持續到2023年,justin531xx,原文標題： 全球塞船運費飆漲 估恐持續到2023年 原文連結： https://reurl...,http://www.ptt.cc/bbs/Stock/M.1641018496.A.7C8...
4,1641053230992_PTT02R,bbs,Ptt,Stock,2022-01-02 00:03:00,[標的] 2610.TW 華航 新年快樂,garyoldb,1. 標的： 2610.TW (例 2330.TW 台積電) 2. 分類：多 3. ...,http://www.ptt.cc/bbs/Stock/M.1641053006.A.2D1...
...,...,...,...,...,...,...,...,...,...
11137,1679387461418_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:19:47,《台北股市》外資21日賣超前十大 中信金淪ATM、貨櫃三雄全沉,時報資訊,【時報-台北電】瑞信危機暫歇，台股21日小幅開高，惟月線反壓尚未突破，加權指數收15513....,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...
11138,1679387460448_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:19:49,《台北股市》三大法人21日賣超前十大 4檔沒「電」遭脫手1萬8000張,時報資訊,【時報-台北電】台股21日加權指數收15513.45點，漲93.48點或0.61%，總成交值...,https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8...
11139,1679391872095_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 16:27:00,電金齊揚，台股收漲93點、站上10日線,Moneydj理財網,MoneyDJ新聞 2023-03-21 16:27:12 記者 新聞中心 報導市場對歐美銀...,https://tw.stock.yahoo.com/news/%E9%9B%BB%E9%8...
11140,1679393432277_YahooStock,news,Yahoo股市,財經新聞,2023-03-21 17:05:06,有風土有歷史有意思 礁溪長榮鳳凰酒店下午茶蘭陽美食攏底佳,姚舜／台北報導,當宜蘭花生捲遇上哈根達斯冰淇淋會擦出什麼火花？礁溪美人腿與古早味金棗迸出鮮滋味！礁溪長榮鳳凰...,https://tw.stock.yahoo.com/news/%E6%9C%89%E9%A...


In [ ]:
news23_ase['post_time'] = pd.to_datetime(news23_ase['post_time']).dt.date
ase['年月日-1'] = ase['年月日'] - datetime.timedelta(days=1)
news23_ase = pd.merge(news23_ase, ase, left_on='post_time', right_on='年月日-1')[['post_time', 'title', 'content', 'label']]
news23_ase

,post_time,title,content,label
0,2022-01-02,[標的] 2610.TW 華航 新年快樂,1. 標的： 2610.TW (例 2330.TW 台積電) 2. 分類：多 3. ...,跌
1,2022-01-02,Re: [請益] 2021台股投報率的中位數不知是幾%?,股票真的很難玩啊～ 壞人很多 像長榮.陽明 公司賺大錢 就讓他慢慢從20漲到140就...,跌
2,2022-01-02,[心得] 超商店員的2021股海奇幻漂流,小弟從高中畢業升大學暑假就在超商打工至今，而賺取的第一份薪水就投入於股市當 中，所以我的股齡...,跌
3,2022-01-02,[心得] 生涯報酬率(第6.5年),附圖是最近一年來的投資帳戶狀況，更早之前資料可參閱之前的生涯文 。不過我發現有些早期的生涯文...,跌
4,2022-01-02,[新聞] 獨》不只台北101！陽明大灑幣 高雄跨年,原文標題： 獨》不只台北101！陽明大灑幣 高雄跨年晚會又見巨幅廣告 原文連結： ht...,跌
...,...,...,...,...
8226,2022-11-27,16檔雙低一高股 凍蒜,工商時報16檔雙低一高股 凍蒜閱讀全文0 陳昱光／台北報導2022年11月27日 上午4:1...,跌
8227,2022-11-27,《台北股市》16檔雙低一高股 凍蒜,時報資訊《台北股市》16檔雙低一高股 凍蒜閱讀全文0 2022年11月27日 上午11:47...,跌
8228,2023-01-08,台驊顏益財：2023年海運驚濤駭浪、有驚無險,台驊控股（2636）董座顏益財7日預估，2023年將是「驚濤駭浪、有驚無險」的一年。貨櫃船運...,跌
8229,2023-01-08,《航運股》台驊顏益財：2023年海運驚濤駭浪、有驚無險,【時報-台北電】台驊控股（2636）董座顏益財7日預估，2023年將是「驚濤駭浪、有驚無險」...,跌


In [ ]:
# 看看content為空的row
c = news23_ase[news23_ase['content'].isna()]
c

,post_time,title,content,label
734,2022-02-16,長榮航上新聞頭版了，恐準備跌一波....,NaN,跌
738,2022-02-16,長榮航上新聞頭版了，恐準備跌一波....,NaN,跌
896,2022-02-21,科技業高薪挖角！長榮航太、華航爆400人離職潮，民航局發聲了,NaN,跌
947,2022-02-22,科技業高薪挖角！長榮航太、華航爆400人離職潮，民航局發聲了,NaN,漲
2867,2022-05-10,長榮委託書大戰啟動 減資生變,NaN,跌
2868,2022-05-10,幸好沒買長榮,NaN,跌
2869,2022-05-10,"為何0056會加入長榮,是因為長榮接下來每年都會配高股息嗎?",NaN,跌
2870,2022-05-10,長榮海2603末升段了嗎?,NaN,跌
2871,2022-05-10,2618長榮航~這總不是主力在出貨的股了~,NaN,跌
2912,2022-05-11,幸好沒買長榮,NaN,跌


In [ ]:
# 去除content為空的row
news23_ase = news23_ase[news23_ase['content'].notna()]

In [ ]:
news23_ase['post_time'] = pd.to_datetime(news23_ase['post_time']).dt.date

In [ ]:
# news_23那邊做完沒有重新排序日期並reset index，所以最後一筆不是接近三月底，訓練資料不是前7312筆、測試資料不是後919筆
train_startDate = datetime.date(2022,1,1)
train_endDate = datetime.date(2022,12,31)

train = news23_ase[news23_ase['post_time'].between(train_startDate, train_endDate)]
train

,post_time,title,content,label
0,2022-01-02,[標的] 2610.TW 華航 新年快樂,1. 標的： 2610.TW (例 2330.TW 台積電) 2. 分類：多 3. ...,跌
1,2022-01-02,Re: [請益] 2021台股投報率的中位數不知是幾%?,股票真的很難玩啊～ 壞人很多 像長榮.陽明 公司賺大錢 就讓他慢慢從20漲到140就...,跌
2,2022-01-02,[心得] 超商店員的2021股海奇幻漂流,小弟從高中畢業升大學暑假就在超商打工至今，而賺取的第一份薪水就投入於股市當 中，所以我的股齡...,跌
3,2022-01-02,[心得] 生涯報酬率(第6.5年),附圖是最近一年來的投資帳戶狀況，更早之前資料可參閱之前的生涯文 。不過我發現有些早期的生涯文...,跌
4,2022-01-02,[新聞] 獨》不只台北101！陽明大灑幣 高雄跨年,原文標題： 獨》不只台北101！陽明大灑幣 高雄跨年晚會又見巨幅廣告 原文連結： ht...,跌
...,...,...,...,...
8223,2022-04-10,《台北股市》外資上周砍殺晶圓雙雄 有檔金融同獲土洋法人大買,【時報-台北電】上周外資持續大賣台股，單周賣超金額高達830億元，觀察外資上周買超仍集中在有...,漲
8224,2022-05-02,興櫃年報樂透 五大天王出爐,興櫃市場2021年財報3日將全數公布，整體稅後淨利上看逾265億元，再創歷史新猷，法人指出，...,漲
8225,2022-05-02,《各報要聞》興櫃年報樂透 五大天王出爐,【時報-台北電】興櫃市場2021年財報3日將全數公布，整體稅後淨利上看逾265億元，再創歷史...,漲
8226,2022-11-27,16檔雙低一高股 凍蒜,工商時報16檔雙低一高股 凍蒜閱讀全文0 陳昱光／台北報導2022年11月27日 上午4:1...,跌


In [ ]:
news23_ase = news23_ase.sort_values(by = 'post_time').reset_index(drop = True) # 日期由小到大排序並reset index

In [ ]:
# news_23重新排序日期並reset index，檢查排序
train_startDate = datetime.date(2022,1,1)
train_endDate = datetime.date(2022,12,31)

train = news23_ase[news23_ase['post_time'].between(train_startDate, train_endDate)]
train

,post_time,title,content,label
0,2022-01-02,[標的] 2610.TW 華航 新年快樂,1. 標的： 2610.TW (例 2330.TW 台積電) 2. 分類：多 3. ...,跌
1,2022-01-02,《金融股》中再保航運股幫打氣,【時報-台北電】中再保（2851）2021年前三季淨利16.53億元，每股盈餘2.8元，獲利...,跌
2,2022-01-02,《熱門族群》3大板塊各有千秋 航運好年冬,【時報-台北電】2021年全年台股上漲3486.31點或漲23.7%，為史上第3大漲點，而海...,跌
3,2022-01-02,《台北股市》台股元月首周 聚焦權值、千金股,【時報-台北電】2021年台股亮麗封關，2022年元月首周盤勢持續挑戰創新高，市場聚焦權值股...,跌
4,2022-01-02,《各報要聞》散戶必學四招 40個月年終自己賺,【時報-台北電】長榮員工爽領40個月、超過百萬元年終獎金的議題，引發全台灣勞工超高的關注度。...,跌
...,...,...,...,...
7296,2022-12-29,[情報] 1229 上市外資買賣超排行,1. 標題：外資買賣超排行 2. 來源：台灣證交所 3. 網址：https://www....,漲
7297,2022-12-29,[情報] 1229 上市投信買賣超排行,1. 標題：投信買賣超排行 2. 來源： 富邦證券 3. 網址：https://fub...,漲
7298,2022-12-29,[新聞] 航空雙雄迎長假商機！ Q1客運量估季增4成,原文標題： 航空雙雄迎長假商機！ Q1客運量估季增4成 原文連結： 發布時間： 2...,漲
7299,2022-12-29,【統一證券晨訊】美科技股賣壓大 中國解封利好觀光餐飲,日期：2022年12月29日※盤勢分析※昨日盤勢美國耶誕節後，外資立刻賣超台股，高價股和IC...,漲


### 存news23_ase.csv

In [ ]:
news23_ase.to_csv('/Users/dora/Desktop/大數據與商業分析/midterm project/news23_ase_0419.csv', index=False, encoding="utf-8")

# 3. 訓練集文章向量化處理

* 首先先進行斷詞處理，以下我們會先實作幾個步驟：
    1. 正則表示法清除多餘字元：先移除文章中符號、英數字，只保留中文字元
    2. 斷句：由於 monpa 在處理 200 字以上字串的斷詞時可能會出現錯誤結果，因此我們統一對長文章先進行斷句拆成較短的句子組成的 list
    3. 斷詞：透過 monpa 對斷句結果中的所有句子進行斷詞

In [ ]:
import monpa
from monpa import utils
import re

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [ ]:
# 這個function用來將字串以正則化處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
with open('/Users/dora/Desktop/大數據與商業分析/Data/stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

### 修改3 (optional) 把tiltle+content合成一個新欄位再斷詞

* 使用 2022 年 1 月至 12 月的文章資料來訓練一個分類器，並用於2023年 1 月至 3 月的股價變動預測當中

分段處理斷詞：2022/1-2022/3

In [ ]:
# 設定訓練資料集的開始日期與結束日期
train_startDate1 = datetime.date(2022,1,1)
train_endDate1 = datetime.date(2022,3,31)

# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list1 = []
for i in list(news23_ase[news23_ase['post_time'].between(train_startDate1, train_endDate1)].index):
    try:
        # 將文章標題和內容合併成一個字串，並進行斷詞處理
        text = clearSentence(news23_ase['title'][i] + '。' + news23_ase['content'][i])
        sentence_list = utils.short_sentence(text)
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list1.append(tokenStr)
    except:
        train_tokenStr_list1.append('')

分段處理斷詞：2022/4-2022/6

In [ ]:
# 設定訓練資料集的開始日期與結束日期
train_startDate2 = datetime.date(2022,4,1)
train_endDate2 = datetime.date(2022,6,30)

# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list2 = []
for i in list(news23_ase[news23_ase['post_time'].between(train_startDate2, train_endDate2)].index):
    try:
        # 將文章標題和內容合併成一個字串，並進行斷詞處理
        text = clearSentence(news23_ase['title'][i] + '。' + news23_ase['content'][i])
        sentence_list = utils.short_sentence(text)
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list2.append(tokenStr)
    except:
        train_tokenStr_list2.append('')


分段處理斷詞：2022/7-2022/9

In [ ]:
# 設定訓練資料集的開始日期與結束日期
train_startDate3 = datetime.date(2022,7,1)
train_endDate3 = datetime.date(2022,9,30)

# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list3 = []
for i in list(news23_ase[news23_ase['post_time'].between(train_startDate3, train_endDate3)].index):
    try:
        # 將文章標題和內容合併成一個字串，並進行斷詞處理
        text = clearSentence(news23_ase['title'][i] + '。' + news23_ase['content'][i])
        sentence_list = utils.short_sentence(text)
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list3.append(tokenStr)
    except:
        train_tokenStr_list3.append('')


分段處理斷詞：2022/10-2022/12

In [ ]:
# 設定訓練資料集的開始日期與結束日期
train_startDate4 = datetime.date(2022,10,1)
train_endDate4 = datetime.date(2022,12,31)

# 透過monpa對文章進行斷詞處理，並將個別斷詞結果寫在tokenStr這個字串變數中，最後再將訓練集中所有字串存在tokenStr_list中
train_tokenStr_list4 = []
for i in list(news23_ase[news23_ase['post_time'].between(train_startDate4, train_endDate4)].index):
    try:
        # 將文章標題和內容合併成一個字串，並進行斷詞處理
        text = clearSentence(news23_ase['title'][i] + '。' + news23_ase['content'][i])
        sentence_list = utils.short_sentence(text)
        tokenStr = str()
        for sentence in sentence_list:
            sentence = clearSentence(sentence)
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        train_tokenStr_list4.append(tokenStr)
    except:
        train_tokenStr_list4.append('')


合併4個list

In [ ]:
train_tokenStr_merged_list = train_tokenStr_list1 + train_tokenStr_list2 + train_tokenStr_list3 + train_tokenStr_list4
train_tokenStr_merged_list

['標的 華航 新年 快樂 標 的 例台積電 分類 多 分析 正文 華航 獲利 與 前景 之 展望 先 就 第四季 簡單 回顧 營收 月 億月 億月 待 公佈 華航 月 有架 貨機 在 檢修 約 影響 成 運能 與 營收 月 單位 收益 月 單位 收益 也 就 是 光貨機 維修 可能 就 影響 了 月億 的 營收 而 月 的 部分 貨機 據 瞭解 月初 已經 正常 所以 營收 上 應該 可 見到 成運能 回升 帶來 的 貢獻 不過 往年 月 因為 聖誕節 急單 結束 單位 收益 多為 小減 今年 在 慢性 塞港 延續 整體 空運價 創 新 高 加上 月 底 砍 航班 影響到 的 腹艙 供給 多重 因素 下 最 保守 的 狀況 下估 應該 不至於 遇到 季節性 運價 大減持平 或 略為 較 月 減少 算是 合理 預期 高於 月 則 是 優於 預期 以 運能 恢復 月 均 運價 持平 的 觀點 來 看 月 營收 有 億 左右 的 機會 應 不 小 這樣 的 話 營收 可能 為 億 左右 依照 前季 的 每 季 固定 成本 以及 油價 漲幅 並 另 考量 可能 的 年終 獎金 費用 後 應該 輕鬆 元 到 元 之間 年底 淨值 約 在 元 左右 多數 券商 仍然 低估 獲利 的 機會 不 小 不過 時序 進入 年 比起 個人 認為 優於 券商 預期 外 股票 是 看 未來 更 重要 的 相信 應該 是 進入 年後 的 個股 展望 此 部分 維持 先前 認為 的 海運格 維持 高檔 將 對 空運 造成 地板價 之 觀點 目前 的 價格 走勢 與 今年 年初 時 相比 運價 在 勢必 仍 將 維持 在 相當 高檔 的 水準 連帶 有 機會 使 空運 的 地板價 維持 在 更 高 價位 不過 具體 來 說 這 個 更 高 價位 可能 是 多少 呢 這 部分 如果 去 觀察 年底 與 年初 的 逐月 空運運價 新聞 運價 應該 差不多 看 新聞 做 股票 會 是 這樣 月運價 月運價 月運價 月運價 月 底 月運價 月運價 月運價 旺季 之後 每 公斤 運價 差不多 跌回 月份 水準 並 在 第一季 底 月 小 旺季 起 開始 回升 這 部分 若 再 往 前 抓 幾 年 抓到 疫情 之前 單純用收 報告 中 的 單位 載貨 收益 來 檢視 的 話 可以 注意到 近年 幾乎 是 相

* 文章向量化處理：我們透過 sklearn 套件中 TfidfVectorizer 將斷詞結果去除停用詞後轉為空間向量

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
pip install --upgrade scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#原始向量
vectorizer = TfidfVectorizer(stop_words=stopwords)
X_train = vectorizer.fit_transform(train_tokenStr_merged_list)
X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names_out())
X_train

,一一一,一一一億,一一三六億,一一九七六,一一九五億,一一九萬,一一二,一一五,一一五三七,一一六四,...,龔招健,龔明鑫,龜仔,龜仔明,龜山,龜山廠,龜甲萬,龜苓膏,龜速,龜速車
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#提取出現頻率排在前 5000 的特徵
#vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=5000)
#X_train = vectorizer.fit_transform(train_tokenStr_merged_list)
#X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names_out())
#X_train

,一二,一半,一口氣,一同,一向,一呂淑美,一年,一度,一成,一日,...,鼓舞,齊揚,齊步,齊漲,齊發,齊跌,龍彩霖,龍頭,龍頭股,龐大
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.055419,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.020594,0.000000,0.0,0.0,0.0,0.024181,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015604,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7307,0.0,0.000000,0.053392,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7308,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7309,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7310,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


* 可以看到使用全部的斷詞結果去組成空間向量時稀疏性會非常大，在後續預測時效率會很低，因此我們需要選擇對分類結果有較顯著影響的詞彙作為向量空間的維度，以下我們透過 Chi-square 計算各詞彙與漲跌標籤的獨立性作為選擇向量空間維度的依據。

In [ ]:
#Chi-square
# 設定訓練資料集的開始日期與結束日期
train_startDate = datetime.date(2022,1,1)
train_endDate = datetime.date(2022,12,31)

y_train = news23_ase[news23_ase['post_time'].between(train_startDate, train_endDate)]['label']

#特徵詞取前1000
chi2_selector = SelectKBest(chi2, k = 1000)
chi2_selector.fit(X_train, y_train)
kbest_vocabs = X_train.columns[chi2_selector.get_support()]
X_train = X_train[kbest_vocabs]
X_train

,一口氣,一模一樣,一監,一萬,三洋實業,上半場,上市股,上揚,上架,上機,...,高爾夫球,鮑爾仍,鮮食,鴻塑,黃南宏,黃文清,黃美賢,黎方國,點位,點收點
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 將 Tfidf 矩陣轉換為 NumPy 陣列
X_train_np = X_train.to_numpy()

# 取得特徵名稱列表
feature_names = list(X_train.columns)

# 將 Tfidf 向量矩陣和特徵名稱保存到 NPZ 文件中
np.savez('/Users/dora/Desktop/大數據與商業分析/Data/vector_file_train_0419.npz', X_train=X_train_np, feature_names=feature_names)


# 4. 測試集文章向量化處理

* 接著我們可以透過 2022/1-2022/12 月的向量維度將 2023/1-2023/3 的文章也轉成 tf-idf 的向量空間

In [ ]:
# 首先先計算3月份的文章個別的tfidf
test_startDate = datetime.date(2023,1,1)
test_endDate = datetime.date(2023,3,21)

test_tokenStr_list = []
for i in list(news23_ase[news23_ase['post_time'].between(test_startDate, test_endDate)].index):
    try:
        txt = clearSentence(news23_ase['title'][i] + '。' + news23_ase['content'][i])
        sentence_list = utils.short_sentence(txt)
        tokenStr = str()
        for sentence in sentence_list:
            tokens = monpa.cut(sentence)
            tokenStr += ' '.join(tokens)
        test_tokenStr_list.append(tokenStr)
    except:
        test_tokenStr_list.append('')

In [ ]:
test_tokenStr_list

['長榮 航勤 請假 潮 稍緩 董事長 等 主管 幫 搬行 李 因為 不滿 年終 獎金 只 有 一 個 月 長榮 航勤 人員 發動 元旦 依法 休假 今天 雖然 自 發性 休假 人數 稍微 減少 但 人力 依舊 不足 就 連 長榮 航勤 董事長 陳有玉 等 一 票 高階 主管 也 下海 支援 幫忙 搬行 李陳有玉 表示 目前 就 是 努力 找 人 回來 上班 找 人 支援 減少 旅客 的 不便 長榮 航勤 員工 此 次 集體 依法 請假 對 出 入境 旅客 造成 衝擊 交通部長 王國材 日 晚 就 親自 致 電 長榮航空 董事長 林寶水 要求 全力 化解 僵局 維護 旅客 權益 據 了解 日 並 沒有 班機 取消 不過有 延誤 分 小時 的 狀況 有 長榮 航勤 員工 透露 長榮 海運 有 傲人 的 年終 長榮 航空 長榮 航太 年終 也 都 有 個 月 長榮 航勤空廚 卻 只 有 個 月 才 會 引發 航勤 人員 不滿 若 公司 沒有 正面 回應 他們 不 排除 農曆 春節 甚至 二二八 連假 繼續 發起 集體 依法 休假 爭取 權益',
 '新聞 長榮海 航運 年終 兩樣 情法人 分析 合 原文 標題 長榮海 航運 年終 兩樣 情法人 分析 合理 差 多 倍 原文 連結 發布 時間 年月日 記者 署名 記者 陳依旻 台北 報導 原文 內容 長榮 海運 日前 宣布 發出 個 月 以上 的 年終 獎金 羨煞 一 票人 但 同 集團 的 長榮 航空 可 就 沒 這麼 讓 人 羨慕 年終 僅 有 個 月 讓 一 堆 員工 在 元旦 假期 主張 依法 休假 不 加 班 同 一 個 集團 為何 年終 行情 差 四十幾 倍 法人 點出 關鍵點 跟 公司 營運 狀況 有關 統一 投顧 董事長 黎方國 指出 長榮 海運光 今年 前 三季 就 賺 超過 億 元 高達 元多億本 上 已經 是 天文 數字 幾乎 賺 了 十幾 個 股 本 年終 分 個 月 算是 合理 但 長榮航 就 沒有 賺 這麼多 前三季 合併 營收 億 元 年增 稅 後 純益 億 元 為 元 以 角度 來 看 長榮航 獲利 是 長榮 海運 的 分之一 只 分 一兩 個 月 的 年終 也 是 合理 至於 為何 長榮海 的 業績 比 長榮航 好 這麼多 黎方國 指出 跟 景氣 有關 通常 成本 比較 高 的 由 航空 

In [ ]:
# 接著將3月份的結果透過df.reindex這個方法映射到訓練集的向量空間中
y_test = news23_ase[news23_ase['post_time'].between(test_startDate, test_endDate)]['label']

vectorizer = TfidfVectorizer(stop_words=stopwords)
X_test = vectorizer.fit_transform(test_tokenStr_list)
X_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names_out())
X_test = X_test.reindex(kbest_vocabs, axis=1, fill_value=0)
X_test

,一口氣,一模一樣,一監,一萬,三洋實業,上半場,上市股,上揚,上架,上機,...,高爾夫球,鮑爾仍,鮮食,鴻塑,黃南宏,黃文清,黃美賢,黎方國,點位,點收點
0,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
1,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.115662,0.0,0
2,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
3,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
4,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
915,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
916,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0
917,0.0,0,0,0,0,0,0.0,0.0,0,0,...,0.0,0,0,0,0.0,0.0,0,0.000000,0.0,0


In [ ]:
# 將 Tfidf 矩陣轉換為 NumPy 陣列
X_test_np = X_test.to_numpy()

# 取得特徵名稱列表
feature_names_test = list(X_test.columns)

# 將 Tfidf 向量矩陣和特徵名稱保存到 NPZ 文件中
np.savez('/Users/dora/Desktop/大數據與商業分析/Data/vector_file_test_0419.npz', X_test=X_test_np, feature_names=feature_names_test)
